<a href="https://colab.research.google.com/github/HSabbar/Generation-lyrics-PNL/blob/master/Text_generation_with_Gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from bs4 import BeautifulSoup
import requests
import time
import re
import sys

In [0]:
def Get_soup_bs4(url):
    requete = requests.get(url)
    page = requete.content
    soup = BeautifulSoup(page, 'html.parser')
    return soup

In [0]:
def find_between(s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [0]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from bs4 import BeautifulSoup
# soup = Get_soup_bs4('https://genius.com/albums/Pnl/Deux-freres')
# soup = Get_soup_bs4('https://genius.com/albums/Pnl/Dans-la-legende')
# soup = Get_soup_bs4('https://genius.com/albums/Pnl/Le-monde-chico')
soup = Get_soup_bs4('https://genius.com/albums/Pnl/Que-la-famille')

urls = []
for url in soup.find_all('a'):
  track = find_between(url.get('href'), 'https://genius.com/Pnl-', '-lyrics')
  if track:
    #print(url.get('href'))
    urls.append(url.get('href'))

In [0]:
urls, len(urls)

(['https://genius.com/Pnl-je-vis-je-visser-lyrics',
  'https://genius.com/Pnl-lala-lyrics',
  'https://genius.com/Pnl-differents-lyrics',
  'https://genius.com/Pnl-obliges-de-prendre-lyrics',
  'https://genius.com/Pnl-de-la-fenetre-au-ter-ter-lyrics',
  'https://genius.com/Pnl-pnl-lyrics',
  'https://genius.com/Pnl-jcomprends-pas-lyrics',
  'https://genius.com/Pnl-gala-gala-lyrics',
  'https://genius.com/Pnl-la-petite-voix-lyrics',
  'https://genius.com/Pnl-athena-lyrics',
  'https://genius.com/Pnl-recherche-du-bonheur-lyrics',
  'https://genius.com/Pnl-simba-lyrics',
  'https://genius.com/Pnl-de-la-fenetre-au-ter-ter-lyrics',
  'https://genius.com/Pnl-athena-lyrics'],
 14)

In [0]:
for url in urls:
  soup_ = Get_soup_bs4(url)
  for i, texte in enumerate(soup_.find_all('p')):
    if i == 0: 
      #All_texte.append(texte.text)
      with open('/content/gdrive/My Drive/Colab Notebooks/PNL-DL/data/Que-la-famille/' + find_between(url, 'Pnl-', '-lyrics') + ".txt", "w") as text_file:
        text_file.write(texte.text)
    else :
      pass

In [0]:
!pip3 install pathlib2

In [0]:
from pathlib2 import Path
def get_ref_files(path : str):
    all_objects = Path(path).glob('**/*.txt')
    files = [str(p) for p in all_objects if p.is_file()]
    return files

In [0]:
REPLACE_NO_CHAR_SPACE = re.compile(r"[-_&;!=\%?\"()\\]")

def load_data(fname):
    with open(fname, 'r') as f:
        text = f.read()
    text = REPLACE_NO_CHAR_SPACE.sub(' ', text.lower())
    text = re.sub(' +', ' ', text)
    data = text.split()
    return data

In [0]:
path_data = '/content/gdrive/My Drive/Colab Notebooks/PNL-DL/data/'
text = []
for path in get_ref_files(path_data) :
  text += load_data(path)

len(get_ref_files(path_data))

71

In [0]:
text[:20]

['[couplet',
 '1',
 'ademo]',
 'ma',
 'frappe',
 "y'a",
 'personne',
 'qui',
 'l’arrête',
 'penalty',
 "j'souris",
 'au',
 'gardien',
 'igo',
 "y'a",
 'mowgli',
 'dans',
 'l’arène',
 'ciseaux,',
 'retournée']

In [0]:
vocab = set(text)
vocab_to_int = {w: idx for idx, w in enumerate(vocab)}
int_to_vocab = {idx: w for idx, w in enumerate(vocab)}

In [0]:
print('Total words: {}'.format(len(text)))
print('Vocab size: {}'.format(len(vocab)))

Total words: 35027
Vocab size: 6617


Convertir le texte en nombre entier

In [0]:
int_text = [vocab_to_int[w] for w in text]

In [0]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
import numpy as np

In [0]:
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow version : {}'.format(tf.__version__))

# GPU
if not tf.test.gpu_device_name():
    warnings.warn('Aucun GPU trouvé, utilisez le GPU pour la projet!')
else:
    print('Périphérique GPU par défaut: {}'.format(tf.test.gpu_device_name()))

TensorFlow version : 1.15.0
Périphérique GPU par défaut: /device:GPU:0


Couche d'entrée

In [0]:
def get_inputs():
    '''
    Création de la couche d'entrée
    '''
    inputs = tf.placeholder(tf.int32, [None, None], name='inputs')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    return inputs, targets, learning_rate

RNN Cell

In [0]:
def get_init_cell(batch_size, rnn_size):
    '''
    Construire des unités RNN empilées
    
    Paramètre
    ---
    batch_size: La taille de chaque batch
    rnn_size: Nombre de neurones cachés dans RNN
    '''
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([lstm])
    
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, 'initial_state')
    return cell, initial_state

Word Embedding


In [0]:

def get_embed(input_data, vocab_size, embed_dim):
    '''
     bcp de mots doivent être intégrés en embedding
     ---
     input_data: tenseur d'entrée
     vocab_size: taille du vocabulaire
     embed_dim: dimension d'intégration
     '''
    embedding = tf.Variable(tf.random_uniform([vocab_size, embed_dim], -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    
    return embed

Build RNN

In [0]:
def build_rnn(cell, inputs):
    '''
    Construire un modèle RNN
    
     Paramètres:
     ---
     cellule: cellule RNN
     entrées: lot d'entrée
    '''
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    
    final_state = tf.identity(final_state, 'final_state')
    return outputs, final_state

Build Neural Network

In [0]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    '''
    Construire un réseau de neurones qui connecte la couche RNN à la couche entièrement connectée
    
     Paramètres:
     ---
     cellule: cellule RNN
     rnn_size: nombre de nœuds RNN cachés
     input_data: tenseur d'entrée
     vocab_size
     embed_dim: incorporer la taille du calque
    
    '''
    embed = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embed)
    
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    
    return logits, final_state

In [0]:
def get_batches(int_text, batch_size, seq_length):
    '''
    Construire batch
    '''
    batch = batch_size * seq_length
    n_batch = len(int_text) // batch
    
    int_text = np.array(int_text[:batch * n_batch]) # Conserver le numéro qui compose un lot complet
    
    int_text_targets = np.zeros_like(int_text)
    int_text_targets[:-1], int_text_targets[-1] = int_text[1:], int_text[0]
    
    # Split
    x = np.split(int_text.reshape(batch_size, -1), n_batch, -1)
    y = np.split(int_text_targets.reshape(batch_size, -1), n_batch, -1)
    
    return np.stack((x, y), axis=1) # Combinaison

In [0]:

# Number de Epochs
num_epochs = 150
# Batch Size
batch_size = 64
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 200
# Sequence Length
seq_length = 20
# Learning Rate
learning_rate = 0.01
# Afficher les statistiques pour chaque n nombre de batch
show_every_n_batches = 100

In [0]:
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab) # vocab_size
    input_text, targets, lr = get_inputs() # Tenseur d'entrée
    input_data_shape = tf.shape(input_text)
    # Initialiser RNN
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # La probabilité de couche softmax
    probs = tf.nn.softmax(logits, name='probs')

    # Fonction de perte
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Fonction d'optimisation
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `layer.__call__` method instead.


In [0]:
batches = get_batches(int_text, batch_size, seq_length)
# Répertoire de stockage des paramètres
save_dir = '/content/gdrive/My Drive/Colab Notebooks/alpha-wann-DL/save'

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # print les résultats à chaque session de formation
            if (epoch * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch,
                    batch_i,
                    len(batches),
                    train_loss))
    # save le modèle
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/27   train_loss = 8.796
Epoch   3 Batch   19/27   train_loss = 2.883
Epoch   7 Batch   11/27   train_loss = 0.459
Epoch  11 Batch    3/27   train_loss = 0.156
Epoch  14 Batch   22/27   train_loss = 0.145
Epoch  18 Batch   14/27   train_loss = 0.111
Epoch  22 Batch    6/27   train_loss = 0.100
Epoch  25 Batch   25/27   train_loss = 0.092
Epoch  29 Batch   17/27   train_loss = 0.107
Epoch  33 Batch    9/27   train_loss = 0.094
Epoch  37 Batch    1/27   train_loss = 0.098
Epoch  40 Batch   20/27   train_loss = 0.114
Epoch  44 Batch   12/27   train_loss = 0.084
Epoch  48 Batch    4/27   train_loss = 0.087
Epoch  51 Batch   23/27   train_loss = 0.084
Epoch  55 Batch   15/27   train_loss = 0.076
Epoch  59 Batch    7/27   train_loss = 0.096
Epoch  62 Batch   26/27   train_loss = 0.091
Epoch  66 Batch   18/27   train_loss = 0.078
Epoch  70 Batch   10/27   train_loss = 0.090
Epoch  74 Batch    2/27   train_loss = 0.094
Epoch  77 Batch   21/27   train_loss = 0.107
Epoch  81 

In [0]:
def get_tensors(loaded_graph):
    '''
    Obtenir les paramètres des résultats de la formation du modèle

    
    Paramètre
    ---
    loaded_graph: Graphique Tensroflow chargé à partir d'un fichier
    '''
    inputs = loaded_graph.get_tensor_by_name('inputs:0')
    initial_state = loaded_graph.get_tensor_by_name('initial_state:0')
    final_state = loaded_graph.get_tensor_by_name('final_state:0')
    probs = loaded_graph.get_tensor_by_name('probs:0')
    return inputs, initial_state, final_state, probs

In [0]:
def pick_word(probabilities, int_to_vocab):
    '''
    Sélectionnez des mots pour la génération de texte pour générer le mot suivant avec une certaine probabilité
    ---
    probabilities: Probabilites du mot suivant
    int_to_vocab: Table de cartographie
    '''
    result = np.random.choice(len(probabilities), 50, p=probabilities)
    return int_to_vocab[result[0]]

In [0]:
# Longueur du texte généré
gen_length = 300

# Définir des mots de démarrage à froid
prime_word = "j'veux la maille comme un roi, solo, j'veux ces bitchs"


result = []
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Charger le modèle
    loader = tf.train.import_meta_graph(save_dir + '.meta')
    loader.restore(sess, save_dir)

    # Obtenir les paramètres des résultats de l'entraînement
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = prime_word.split()
    #print(gen_sentences)
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Générer des phrases

    for n in range(gen_length):

        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Prévision
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})

        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab) 
        gen_sentences.append(pred_word)
        #print(pred_word)
        result.append(pred_word + ' ')


INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/Colab Notebooks/alpha-wann-DL/save


In [0]:
lyrics = ''.join(result)
lyrics = lyrics.replace(',', '\n')
new_line = []
for line in lyrics.split('\n'):

  between = find_between(line, '[', ']')
  if between : 
    ren = line.split('['+between+']')
    print(ren[0]+'\n'+'['+between+']'+'\n'+ren[1])
  else:
    for l in line.split():
      new_line.append(l)
      if len(new_line) > 15:
        print(' '.join(new_line) )
        new_line = []

 igo t'es dans la merde dans le benda j'sais que j'm'enfonce plus sur ma gueule veut sa mère la reine un sourire des des doss' des milliers j'veux plus je me va 
[couplet 1 : n.o.s]
 si belle la on la merde : faut la défaite n'est ça me faut pas mon égal
en j'ai d'la à ta femme viens voir tout à la vie faire au fond d’un
hall ça y est a qu'à t'a qu'au drôle d'attitude man a a a abonné a
a abonné a a a abonné a a abonné a pas ma putain la p'tite souris
cette ‘tasse regarde le temps c'est la vérité j'reprends tu sors à chaque bon char la
guerre guerre l'oseille et tout trop vite fait fait j’emballe les miens sont loin qui m'ont
du sale j'ai toujours des voit de putes de j lopez de larmes : ma chair
 le rain ter 
[couplet 2 : ademo]
 dans mon igo de ces bâtards
vaut ce soir j’sors si de balade ce et en veut encore ouais mon tu connais
l'adresse rien dans ma le même que le taga remonté d'malaga sheitana plus veut pas envie
d'rigoler pourtant elle la plus la teille que la mif que la mif que

In [0]:
lyrics = ''.join(result)
lyrics = lyrics.replace(',', '\n')
new_line = []
for line in lyrics.split('\n'):

  between = find_between(line, '[', ']')
  if between : 
    ren = line.split('['+between+']')
    print(ren[0]+'\n'+'['+between+']'+'\n'+ren[1])
  else:
    for l in line.split():
      new_line.append(l)
      if len(new_line) > 15:
        print(' '.join(new_line) )
        new_line = []



[pont : ademo]
 chang
chang chang j'marche c'est le boule j'ai des fois j't'en veux d'pas avoir été là j'suis
plus là pour être honnête double j'ai les depuis cette petite voix on va tout à
 tout à l'envers 
[couplet 1 : n.o.s]
 faut que j'apprenne et pour la hassana le sun me l'odeur d'l'argent ta haine qui m'a apaisé paisé
l'envers à l'envers la terre tourne j'me contourne j'vois tout à l'envers paisé pas tes chaînes
igo les tits pe moi qui sais c'que j'sais pas j'suis heure pas comme eux pas
 pas la plus pas là j'fais l'tour la de la fenêtre au ter ter petit frère ira chercher l'oseille de la fenêtre au ter ter 
[couplet 3 : ademo]
 j'fume deux frères
comme eux pas comme eux le m en l'air dit mon cœur en tu viens manny
soit pas pourquoi pas envie d'rigoler quand je t'aime ouais ouais ouais des pédés ou l'argent
ouais une y'a rien en essonne toi tu la mif' d'abord c'est quand du bien je
suis rassuré ma putain d'tête j'ai de le soir j'ai des hommes le soir j'rentre à
la maison j'pose